In [1]:
import tensorflow as tf
from tensorflow.contrib.slim.nets import inception
import tensorflow.contrib.slim as slim
import numpy as np
from flower_model import FlowerModel

flower_model = FlowerModel()
flower_model.fetch()
labels = flower_model.get_class_labels()

print('class: {}'.format(labels))

class: ['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']


In [2]:
height, width, n_channels = (299, 299, 3)

X = tf.placeholder(tf.float32, shape=[None, height, width, n_channels])
training = tf.placeholder_with_default(False, shape=[])

with slim.arg_scope(inception.inception_v3_arg_scope()):
    logits, end_points = inception.inception_v3(X, num_classes=1001, is_training=training)

inception_saver = tf.train.Saver()

In [3]:
for layer, operation in end_points.items():
    print('-'*80)
    print('Layer=', layer)
    print('Operation=', operation)

--------------------------------------------------------------------------------
Layer= Conv2d_1a_3x3
Operation= Tensor("InceptionV3/InceptionV3/Conv2d_1a_3x3/Relu:0", shape=(?, 149, 149, 32), dtype=float32)
--------------------------------------------------------------------------------
Layer= Conv2d_2a_3x3
Operation= Tensor("InceptionV3/InceptionV3/Conv2d_2a_3x3/Relu:0", shape=(?, 147, 147, 32), dtype=float32)
--------------------------------------------------------------------------------
Layer= Conv2d_2b_3x3
Operation= Tensor("InceptionV3/InceptionV3/Conv2d_2b_3x3/Relu:0", shape=(?, 147, 147, 64), dtype=float32)
--------------------------------------------------------------------------------
Layer= MaxPool_3a_3x3
Operation= Tensor("InceptionV3/InceptionV3/MaxPool_3a_3x3/MaxPool:0", shape=(?, 73, 73, 64), dtype=float32)
--------------------------------------------------------------------------------
Layer= Conv2d_3b_1x1
Operation= Tensor("InceptionV3/InceptionV3/Conv2d_3b_1x1/Relu:0

In [4]:
prelogits = tf.squeeze(end_points['PreLogits'], axis=[1,2])
print(prelogits)

Tensor("Squeeze:0", shape=(?, 2048), dtype=float32)


In [5]:
# Concatanate DNN layers to classify flower classes.
n_outputs = len(labels)

with tf.name_scope('flower_classifier_dnn'):
    flower_logits = tf.layers.dense(prelogits, n_outputs, name='flower_logits')
    y_prob = tf.nn.softmax(flower_logits, name='y_prob')

y = tf.placeholder(tf.int32, shape=[None])

with tf.name_scope('train'):
    # Train only new DNN layers, others are frozen.
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=flower_logits, labels=y)
    loss = tf.reduce_mean(xentropy)
    optimizer = tf.train.AdamOptimizer()
    flower_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='flower_logits')
    training_op = optimizer.minimize(loss, var_list=flower_vars)

with tf.name_scope('eval'):
    correct = tf.nn.in_top_k(flower_logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
with tf.name_scope('init_and_save'):
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()

W0902 07:33:14.991181 139634323130176 deprecation.py:323] From <ipython-input-5-f53cee0fbe64>:5: dense (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.dense instead.
W0902 07:33:14.994246 139634323130176 deprecation.py:506] From /home/guhwanbae/anaconda3/envs/gu-keras/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [6]:
test_size = 0.2
X_train, X_test, y_train, y_test = flower_model.train_test_split(test_size)

In [7]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler((-1.0, 1.0), copy=False)

original_shape = X_train.shape
X_train = scaler.fit_transform(X_train.reshape(-1, original_shape[-1]))
X_train = X_train.reshape(original_shape)

In [8]:
n_epoch = 40

n_samples = len(X_train)
batch_size = 40
aug_size = 5

In [9]:
from inception_model_downloader import InceptionModelDownloader

model_fetcher = InceptionModelDownloader()
model_fetcher.fetch_inception_v3()

In [ ]:
import image_transform

with tf.Session() as sess:
    init.run()
    inception_saver.restore(sess, model_fetcher.checkpoint_path)
    
    for epoch in range(n_epoch):
        print('Epoch : ', epoch, end='')
        for X_batch, y_batch in flower_model.random_batch(X_train, y_train, batch_size,
                                        augmentation_op=image_transform.random_transform,
                                        aug_size=aug_size):
            print('.', end='')
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch, training: True})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        print('acc_train :', acc_train)
            
        save_path = saver.save(sess, './flower_classifier_with_inceptionv3')

W0902 07:33:41.401391 139634323130176 deprecation.py:323] From /home/guhwanbae/anaconda3/envs/gu-keras/lib/python3.7/site-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


Epoch :  0..........................................................................acc_train : 0.51794875
Epoch :  1..........................................................................acc_train : 0.53846157
Epoch :  2..........................................................................acc_train : 0.574359
Epoch :  3..........................................................................acc_train : 0.73333335
Epoch :  4..........................................................................acc_train : 0.5897436
Epoch :  5..........................................................................acc_train : 0.71794873
Epoch :  6..........................................................................acc_train : 0.6051282
Epoch :  7..........................................................................acc_train : 0.5692308
Epoch :  8..........................................................................acc_train : 0.5897436
Epoch :  9.................................

In [ ]:
original_test_shape = X_test.shape
X_test = scaler.transform(X_test.reshape(-1, original_test_shape[-1]))
X_test = X_test.reshape(original_test_shape)

In [ ]:
n_test_batches = 10
X_test_batches = np.array_split(X_test, n_test_batches)
y_test_batches = np.array_split(y_test, n_test_batches)

with tf.Session() as sess:
    saver.restore(sess, './flower_classifier_with_inceptionv3')
    acc_test = np.mean([accuracy.eval(feed_dict={X: X_test_batch, y: y_test_batch})
                        for X_test_batch, y_test_batch in zip(X_test_batches, y_test_batches)])
    print('acc_test : ', acc_test)